## My work for predicting housing prices TRAZZ PEPPER

This is my work for using a kaggle dataset and beging to try and understand the relationships between the feature varaibles and the final price of the house. I have gone through and commeted extensivly to try and explain what I am doing and why I am doing what I am doing.
The score I got on kaggle was 0.12945. (place 776)

In [99]:
# The main packages I used are pandas, numpy and sklearn.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error

In [100]:
# The data was originally in a csv (train.csv)
path = "train.csv"
df = pd.read_csv(path)

In [101]:
# This is the function for data conditioning, this allows me formate the data for both training and testing the same.
# The first part turns categorical variables into dummies. (boolean values)


def data_conditioning(dataframe):
    dataframe.set_index("Id", inplace=True)
    categorical_columns = dataframe.select_dtypes(include=["object"]).columns

    for col in categorical_columns:
        dataframe = dataframe.drop(
            [col],
            axis=1,
        ).join(pd.get_dummies(df[col], prefix=f"{col}"))

    # Feature Engineering stuff I am trying, they make new variables from certian features
    # Turns Overall Quality into boolean as Overall quality was a numerical categorical variable.
    dataframe["Qual_Very_Excellent"] = (dataframe["OverallQual"] == 10).astype(int)
    dataframe["Qual_Excellent"] = (dataframe["OverallQual"] == 9).astype(int)
    dataframe["Qual_Very_Good"] = (dataframe["OverallQual"] == 8).astype(int)
    dataframe["Qual_Good"] = (dataframe["OverallQual"] == 7).astype(int)
    dataframe["Qual_Above_Average"] = (dataframe["OverallQual"] == 6).astype(int)
    dataframe["Qual_Average"] = (dataframe["OverallQual"] == 5).astype(int)
    dataframe["Qual_Below_Average"] = (dataframe["OverallQual"] == 4).astype(int)
    dataframe["Qual_Fair"] = (dataframe["OverallQual"] == 3).astype(int)
    dataframe["Qual_Poor"] = (dataframe["OverallQual"] == 2).astype(int)
    dataframe["Qual_Very_Poor"] = (dataframe["OverallQual"] == 1).astype(int)
    # dataframe = dataframe.drop(["OverallQual"], axis=1)

    # Turns Overall Condition into boolean as Overall condition was a numerical categorical variable.
    dataframe["Cond_Very_Excellent"] = (dataframe["OverallCond"] == 10).astype(int)
    dataframe["Cond_Excellent"] = (dataframe["OverallCond"] == 9).astype(int)
    dataframe["Cond_Very_Good"] = (dataframe["OverallCond"] == 8).astype(int)
    dataframe["Cond_Good"] = (dataframe["OverallCond"] == 7).astype(int)
    dataframe["Cond_Above_Average"] = (dataframe["OverallCond"] == 6).astype(int)
    dataframe["Cond_Average"] = (dataframe["OverallCond"] == 5).astype(int)
    dataframe["Cond_Below_Average"] = (dataframe["OverallCond"] == 4).astype(int)
    dataframe["Cond_Fair"] = (dataframe["OverallCond"] == 3).astype(int)
    dataframe["Cond_Poor"] = (dataframe["OverallCond"] == 2).astype(int)
    dataframe["Cond_Very_Poor"] = (dataframe["OverallCond"] == 1).astype(int)
    # dataframe = dataframe.drop(["OverallCond"], axis=1)

    # Turns MSSubClass into boolean as MSSubClass was a numerical categorical variable.
    dataframe["1-STORY 1946 & NEWER ALL STYLES"] = (
        dataframe["MSSubClass"] == 20
    ).astype(int)
    dataframe["1-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 30).astype(int)
    dataframe["1-STORY W/FINISHED ATTIC ALL AGES"] = (
        dataframe["MSSubClass"] == 40
    ).astype(int)
    dataframe["1-1/2 STORY - UNFINISHED ALL AGES"] = (
        dataframe["MSSubClass"] == 45
    ).astype(int)
    dataframe["1-1/2 STORY FINISHED ALL AGES"] = (dataframe["MSSubClass"] == 50).astype(
        int
    )
    dataframe["2-STORY 1946 & NEWER"] = (dataframe["MSSubClass"] == 60).astype(int)
    dataframe["2-STORY 1945 & OLDER"] = (dataframe["MSSubClass"] == 70).astype(int)
    dataframe["2-1/2 STORY ALL AGES"] = (dataframe["MSSubClass"] == 75).astype(int)
    dataframe["SPLIT OR MULTI-LEVEL"] = (dataframe["MSSubClass"] == 80).astype(int)
    dataframe["SPLIT FOYER"] = (dataframe["MSSubClass"] == 85).astype(int)
    dataframe["DUPLEX - ALL STYLES AND AGES"] = (dataframe["MSSubClass"] == 90).astype(
        int
    )
    dataframe["1-STORY PUD (Planned Unit Development) - 1946"] = (
        dataframe["MSSubClass"] == 120
    ).astype(int)
    dataframe["1-1/2 STORY PUD - ALL AGES"] = (dataframe["MSSubClass"] == 150).astype(
        int
    )
    dataframe["2-STORY PUD - 1946 & NEWER"] = (dataframe["MSSubClass"] == 160).astype(
        int
    )
    dataframe["PUD - MULTILEVEL - INCL SPLIT LEV/FOYER"] = (
        dataframe["MSSubClass"] == 180
    ).astype(int)
    dataframe["2 FAMILY CONVERSION - ALL STYLES AND AGES"] = (
        dataframe["MSSubClass"] == 190
    ).astype(int)
    dataframe = dataframe.drop(["MSSubClass"], axis=1)

    # Making some new features too see if they better correlated with the dataframe.
    # 1. Squared values of TotalBsmtSF and LotFrontage
    dataframe["TotalBsmtSF_squared_FE"] = dataframe["TotalBsmtSF"] ** 2
    dataframe["LotFrontage_squared_FE"] = dataframe["LotFrontage"] ** 2

    # 2. Interaction terms between TotalBsmtSF and GarageArea
    dataframe["TotalBsmtSF_GarageArea_FE"] = (
        dataframe["TotalBsmtSF"] * dataframe["GarageArea"]
    )

    # 3. Age of the house (years since built)
    dataframe["YearBuilt_age_FE"] = 2024 - dataframe["YearBuilt"]

    # 4. Total number of bathrooms (FullBath + HalfBath/2 + BsmtFullBath)
    dataframe["Total_Baths_FE"] = (
        dataframe["FullBath"] + (dataframe["HalfBath"] / 2) + dataframe["BsmtFullBath"]
    )

    # 5. GarageArea to Total Area (GrLivArea + TotalBsmtSF) ratio
    dataframe["GarageArea_to_TotalArea_FE"] = dataframe["GarageArea"] / (
        dataframe["GrLivArea"] + dataframe["TotalBsmtSF"] + 1e-5
    )

    # 6. GrLivArea to GarageArea ratio
    dataframe["GrLivArea_to_GarageArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["GarageArea"] + 1e-5
    )

    # 7. TotalBsmtSF to GarageArea ratio
    dataframe["TotalBsmtSF_to_GarageArea_FE"] = dataframe["TotalBsmtSF"] / (
        dataframe["GarageArea"] + 1e-5
    )

    # 8. Years since renovation (2024 - YearRemodAdd)
    dataframe["Years_Since_Renovation_FE"] = 2024 - dataframe["YearRemodAdd"]

    # 9. Total number of rooms (above ground rooms + 1 if basement GLQ is finished)
    dataframe["TotalRooms_FE"] = dataframe["TotRmsAbvGrd"] + (
        dataframe["BsmtFinType1_GLQ"] > 0
    ).astype(int)

    # 10. LotArea to GarageCars ratio
    dataframe["LotArea_to_GarageCars_FE"] = dataframe["LotArea"] / (
        dataframe["GarageCars"] + 1e-5
    )

    # 11. Total square footage (GrLivArea + TotalBsmtSF + 1stFlrSF + 2ndFlrSF)
    dataframe["Total_SF_FE"] = (
        dataframe["GrLivArea"]
        + dataframe["TotalBsmtSF"]
        + dataframe["1stFlrSF"]
        + dataframe["2ndFlrSF"]
    )

    # 12. Total number of bedrooms (above ground bedrooms + 1 if basement BLQ or GLQ is finished)
    dataframe["Total_Bedrooms_FE"] = (
        dataframe["BedroomAbvGr"]
        + (dataframe["BsmtFinType1_BLQ"] > 0).astype(int)
        + (dataframe["BsmtFinType1_GLQ"] > 0).astype(int)
    )

    # 13. Garage SF to Total SF ratio
    dataframe["Garage_SF_to_Total_SF_FE"] = dataframe["GarageArea"] / (
        dataframe["Total_SF_FE"] + 1e-5
    )

    # 14. GrLivArea to LotArea ratio
    dataframe["GrLivArea_to_LotArea_FE"] = dataframe["GrLivArea"] / (
        dataframe["LotArea"] + 1e-5
    )

    # 15. GarageCars to TotalRooms ratio
    dataframe["GarageCars_to_TotalRooms_FE"] = dataframe["GarageCars"] / (
        dataframe["TotalRooms_FE"] + 1e-5
    )

    # 16. Outdoor space (sum of WoodDeckSF, OpenPorchSF, EnclosedPorch, ScreenPorch)
    dataframe["Outdoor_Space_FE"] = (
        dataframe["WoodDeckSF"]
        + dataframe["OpenPorchSF"]
        + dataframe["EnclosedPorch"]
        + dataframe["ScreenPorch"]
    )
    dataframe["Area_per_Car_FE"] = dataframe["GarageArea"] / (
        dataframe["GarageCars"] + 1e-5
    )
    dataframe["QualCond_Merge_FE"] = dataframe["OverallQual"] + dataframe["OverallCond"]

    dataframe["LotArea_to_LotFrontage_FE"] = dataframe["LotArea"] / (
        dataframe["LotFrontage"] + 1e-5
    )

    dataframe = dataframe.drop(["OverallQual"], axis=1)
    dataframe = dataframe.drop(["OverallCond"], axis=1)

    # Fills missing data with median (better ways to do this on improvement)
    dataframe.fillna(dataframe.median(), inplace=True)

    dataframe.columns = dataframe.columns.str.replace(" ", "_")

    return dataframe

In [102]:
# Data contitions and fills with median for missing values, and then scales the data
df = data_conditioning(dataframe=df)


# These columns are not in the test dataset so drop them
columns_to_drop = [
    "Condition2_RRAe",
    "Condition2_RRAn",
    "Condition2_RRNn",
    "Electrical_Mix",
    "Exterior1st_ImStucc",
    "Exterior1st_Stone",
    "Exterior2nd_Other",
    "GarageQual_Ex",
    "Heating_Floor",
    "Heating_OthW",
    "HouseStyle_2.5Fin",
    "MiscFeature_TenC",
    "PoolQC_Fa",
    "RoofMatl_ClyTile",
    "RoofMatl_Membran",
    "RoofMatl_Metal",
    "RoofMatl_Roll",
    "Utilities_NoSeWa",
]

df = df.drop(columns=columns_to_drop)


# Separate features and target variable
X = df.drop(columns=["SalePrice"], axis=1)
y = df["SalePrice"]

In [103]:
# Correlation Matrix to choose which variables to focus on. (however, correlation matrix too big so output a file better)
correlation_matrix = df.corr()
saleprice_correlation = (
    correlation_matrix["SalePrice"].abs().sort_values(ascending=False)
)

# Keep the variabels with correlation of 0.005 or higher, gives better output for gradient boosting.
high_corr_vars_list = saleprice_correlation[saleprice_correlation > 0.05].index.tolist()

# Just keeps highly correlated variables.
high_corr_vars = df[high_corr_vars_list]
del high_corr_vars_list[0]
X = high_corr_vars.drop(columns=["SalePrice"], axis=1)
y = high_corr_vars["SalePrice"]

In [105]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_squared_log_error

# Scales the data all the same
scaler = StandardScaler()
X_S = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# PCA feature selection, however was not helpful
# pca = PCA(n_components=0.95)
# X_pca = pca.fit_transform(X_S)


# Training and Testing
X_train, X_test, y_train, y_test = train_test_split(
    X_S, y, test_size=0.2, random_state=1
)

# Basic Linear Regression Model to see how good we start.
reg = LinearRegression()


reg.fit(X_train, y_train)
predictions_linear = reg.predict(X_test)

print(f"The linear model fits to training data: {round(reg.score(X_train, y_train),3)}")
print(f"The linear model fits to testing data: {round(reg.score(X_test, y_test),3)}")
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, predictions_linear))}"
)

The linear model fits to training data: 0.916
The linear model fits to testing data: 0.87
The negative MSLE is 0.16656253914050884


In [108]:
import optuna
from optuna.samplers import TPESampler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
import numpy as np


# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
    max_depth = trial.suggest_int("max_depth", 10, 70, step=6)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 5)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    min_weight_fraction_leaf = trial.suggest_float(
        "min_weight_fraction_leaf", 1e-6, 0.4, log=True
    )
    max_leaf_nodes = trial.suggest_int("max_leaf_nodes", 15, 575, step=40)
    max_samples = trial.suggest_float("max_samples", 0.2, 1.0, log=True)

    # Initialize and train the RandomForestRegressor
    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        min_weight_fraction_leaf=min_weight_fraction_leaf,
        max_leaf_nodes=max_leaf_nodes,
        max_samples=max_samples,
        n_jobs=-1,
        verbose=0,
    )

    model.fit(X_train, y_train)

    # Return the model score for Optuna to minimize the objective
    model_prediction = model.predict(X_test)
    return np.sqrt(mean_squared_log_error(y_test, model_prediction))


# Set up Optuna study
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=150)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# Evaluate the best model
best_model_RF = RandomForestRegressor(**best_params, n_jobs=-1)
best_model_RF.fit(X_train, y_train)
test_score = best_model_RF.score(X_test, y_test)
predictions_forest = best_model_RF.predict(X_test)

print(f"Test Score: {test_score}")
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, predictions_forest))}"
)

[I 2024-10-19 14:17:33,730] A new study created in memory with name: no-name-9c53b0ea-dcc7-4cb4-88a1-016fbda89107
[I 2024-10-19 14:17:34,467] Trial 0 finished with value: 0.17788209364812366 and parameters: {'n_estimators': 400, 'max_depth': 70, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'min_weight_fraction_leaf': 1.5246513340955065e-05, 'max_leaf_nodes': 175, 'max_samples': 0.2005007490464868}. Best is trial 0 with value: 0.17788209364812366.
[I 2024-10-19 14:17:36,198] Trial 1 finished with value: 0.18638367317037338 and parameters: {'n_estimators': 500, 'max_depth': 34, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'min_weight_fraction_leaf': 0.002901562571818266, 'max_leaf_nodes': 255, 'max_samples': 0.20310680376162385}. Best is trial 0 with value: 0.17788209364812366.
[I 2024-10-19 14:17:36,927] Trial 2 finished with value: 0.17790110807687354 and parameters: {'n_estimators': 400, 'max_depth': 40, 'min_samples_split': 4, 'min_

Best Parameters: {'n_estimators': 300, 'max_depth': 28, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': None, 'min_weight_fraction_leaf': 3.910136241591091e-06, 'max_leaf_nodes': 575, 'max_samples': 0.9493555349033783}
Test Score: 0.8887258592137264
The negative MSLE is 0.14424707683927593


In [92]:
from lightgbm import LGBMRegressor


# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    num_leaves = trial.suggest_int("num_leaves", 32, 298, step=7)
    reg_lambda = trial.suggest_float("reg_lambda", 1e-5, 4, log=True)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-5, 4, log=True)
    n_estimators = trial.suggest_int("n_estimators", 50, 330, step=40)
    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 15, 55, step=4)
    min_split_gain = trial.suggest_float("min_split_gain", 1e-8, 0.1, log=True)
    min_child_weight = trial.suggest_float("min_child_weight", 1e-4, 0.1, log=True)
    min_child_samples = trial.suggest_int("min_child_samples", 14, 30, step=4)
    subsample = trial.suggest_float("subsample", 0.25, 1, log=True)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.25, 1, log=True)

    # Initialize and train the LGBMRegressor
    model = LGBMRegressor(
        num_leaves=num_leaves,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_split_gain=min_split_gain,
        min_child_weight=min_child_weight,
        min_child_samples=min_child_samples,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        n_jobs=-1,
        verbose=-1,
    )

    model.fit(X_train, y_train)

    # Return the model score for Optuna to minimize the objective
    model_prediction = model.predict(X_test)
    return np.sqrt(mean_squared_log_error(y_test, model_prediction))


# Set up the Optuna study
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=400)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# Train the best model
best_model_LGBM = LGBMRegressor(**best_params, n_jobs=-1, verbose=-1)
best_model_LGBM.fit(X_train, y_train)

# Evaluate the best model
test_score_LGBM = best_model_LGBM.score(X_test, y_test)
predictions_LGBM = best_model_LGBM.predict(X_test)

print(f"Test Score: {test_score_LGBM}")
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, predictions_LGBM))}"
)

[I 2024-10-15 16:26:53,975] A new study created in memory with name: no-name-b1422156-03a8-467c-849e-0e4e8ef6c1f3
[I 2024-10-15 16:26:55,781] Trial 0 finished with value: 0.1371730831264365 and parameters: {'num_leaves': 207, 'reg_lambda': 0.003500975168993703, 'reg_alpha': 0.0030763583093812185, 'n_estimators': 250, 'learning_rate': 0.039174149641665455, 'max_depth': 55, 'min_split_gain': 1.6969377381100543e-08, 'min_child_weight': 0.03903821535724627, 'min_child_samples': 18, 'subsample': 0.5282923111332966, 'colsample_bytree': 0.6837955285301291}. Best is trial 0 with value: 0.1371730831264365.
[I 2024-10-15 16:26:55,993] Trial 1 finished with value: 0.14698360087472195 and parameters: {'num_leaves': 32, 'reg_lambda': 0.6975364467587345, 'reg_alpha': 0.2630809905716852, 'n_estimators': 90, 'learning_rate': 0.03163170064683021, 'max_depth': 39, 'min_split_gain': 4.014906947459366e-06, 'min_child_weight': 0.07971181514815852, 'min_child_samples': 26, 'subsample': 0.9332693143518452, '

Best Parameters: {'num_leaves': 207, 'reg_lambda': 3.96617428140741, 'reg_alpha': 2.305573270321288e-05, 'n_estimators': 250, 'learning_rate': 0.03983336321503316, 'max_depth': 31, 'min_split_gain': 1.9774493702956673e-07, 'min_child_weight': 0.030011794036179984, 'min_child_samples': 30, 'subsample': 0.5197390152347325, 'colsample_bytree': 0.5719674560463301}
Test Score: 0.8947969678339945
The negative MSLE is 0.1255309924394462


In [93]:
from catboost import CatBoostRegressor


# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    iterations = trial.suggest_int("iterations", 750, 2500, step=250)
    learning_rate = trial.suggest_float("learning_rate", 0.005, 0.1, log=True)
    l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 0.8, 50, log=True)
    bagging_temperature = trial.suggest_float("bagging_temperature", 1e-4, 1, log=True)
    random_strength = trial.suggest_int("random_strength", 1, 10)
    depth = trial.suggest_int("depth", 6, 10)
    min_data_in_leaf = trial.suggest_int("min_data_in_leaf", 1, 4)
    border_count = trial.suggest_int("border_count", 24, 274, step=50)

    # Initialize and train the CatBoostRegressor
    model = CatBoostRegressor(
        iterations=iterations,
        learning_rate=learning_rate,
        l2_leaf_reg=l2_leaf_reg,
        bagging_temperature=bagging_temperature,
        random_strength=random_strength,
        depth=depth,
        min_data_in_leaf=min_data_in_leaf,
        border_count=border_count,
        verbose=0,
    )

    model.fit(X_train, y_train)

    # Return the model score for Optuna to minimize the objective
    model_prediction = model.predict(X_test)
    return np.sqrt(mean_squared_log_error(y_test, model_prediction))


# Set up the Optuna study
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=30)

# Get the best hyperparameters
best_params = study.best_params
print(f"Best Parameters: {best_params}")

# Train the best model
best_model_catboost = CatBoostRegressor(**best_params, verbose=0)
best_model_catboost.fit(X_train, y_train)

# Evaluate the best model
test_score_catboost = best_model_catboost.score(X_test, y_test)
predictions_catboost = best_model_catboost.predict(X_test)
print(f"Test Score: {test_score_catboost}")
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, predictions_catboost))}"
)

[I 2024-10-15 16:32:30,584] A new study created in memory with name: no-name-7d031c1f-ff3a-4093-bbc5-7931d9f6bfa3
[I 2024-10-15 16:32:40,607] Trial 0 finished with value: 0.12365711860838222 and parameters: {'iterations': 2250, 'learning_rate': 0.024179514817570826, 'l2_leaf_reg': 8.386936627329122, 'bagging_temperature': 0.0012120203054299154, 'random_strength': 1, 'depth': 7, 'min_data_in_leaf': 4, 'border_count': 224}. Best is trial 0 with value: 0.12365711860838222.
[I 2024-10-15 16:32:45,756] Trial 1 finished with value: 0.12626090400807538 and parameters: {'iterations': 2000, 'learning_rate': 0.08015570116918998, 'l2_leaf_reg': 1.1135873460658583, 'bagging_temperature': 0.6175664384514657, 'random_strength': 9, 'depth': 6, 'min_data_in_leaf': 1, 'border_count': 174}. Best is trial 0 with value: 0.12365711860838222.
[I 2024-10-15 16:33:35,846] Trial 2 finished with value: 0.13806179078681585 and parameters: {'iterations': 2000, 'learning_rate': 0.0931610773317068, 'l2_leaf_reg': 1

Best Parameters: {'iterations': 2250, 'learning_rate': 0.021659142658350786, 'l2_leaf_reg': 4.222261901606465, 'bagging_temperature': 0.0009853660146216681, 'random_strength': 4, 'depth': 6, 'min_data_in_leaf': 3, 'border_count': 74}
Test Score: 0.8957856011950993
The negative MSLE is 0.11880075533049309


In [94]:
from xgboost import XGBRegressor


# Define the objective function for Optuna
# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters for XGBoost
    n_estimators = trial.suggest_int("n_estimators", 100, 500, step=100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.1, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 8, step=2)
    min_child_weight = trial.suggest_int("min_child_weight", 1, 5)
    gamma = trial.suggest_float("gamma", 0.001, 1, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1)
    reg_alpha = trial.suggest_float("reg_alpha", 1e-4, 1, log=True)
    reg_lambda = trial.suggest_float("reg_lambda", 0.1, 10, log=True)
    scale_pos_weight = trial.suggest_int("scale_pos_weight", 1, 10)
    booster = trial.suggest_categorical("booster", ["gbtree", "dart"])
    tree_method = trial.suggest_categorical("tree_method", ["hist"])
    grow_policy = trial.suggest_categorical("grow_policy", ["depthwise"])

    # Initialize and train the XGBRegressor
    model = XGBRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_child_weight=min_child_weight,
        gamma=gamma,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        scale_pos_weight=scale_pos_weight,
        booster=booster,
        tree_method=tree_method,
        grow_policy=grow_policy,
        n_jobs=-1,
    )

    model.fit(X_train, y_train)

    # Return the model score for Optuna to minimize the objective
    model_prediction = model.predict(X_test)
    return np.sqrt(mean_squared_log_error(y_test, model_prediction))


# Set up the Optuna study
study = optuna.create_study(direction="minimize", sampler=TPESampler())
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params_xgb = study.best_params
print(f"Best Parameters: {best_params_xgb}")

# Train the best model
best_model_xgb = XGBRegressor(**best_params_xgb, verbosity=0, n_jobs=-1)
best_model_xgb.fit(X_train, y_train)

# Evaluate the best model
test_score_xgb = best_model_xgb.score(X_test, y_test)
predictions_xgb = best_model_xgb.predict(X_test)
print(f"Test Score: {test_score_xgb}")
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, predictions_xgb))}"
)

[I 2024-10-15 16:44:23,675] A new study created in memory with name: no-name-d7f889bb-e022-4715-ba0b-4101d2626a69
[I 2024-10-15 16:44:24,556] Trial 0 finished with value: 0.13172927322785893 and parameters: {'n_estimators': 400, 'learning_rate': 0.059776937404007816, 'max_depth': 2, 'min_child_weight': 3, 'gamma': 0.033478352976227616, 'subsample': 0.9686705807027225, 'colsample_bytree': 0.6189705490469558, 'reg_alpha': 0.030346963561066215, 'reg_lambda': 0.41262762861156455, 'scale_pos_weight': 9, 'booster': 'gbtree', 'tree_method': 'hist', 'grow_policy': 'depthwise'}. Best is trial 0 with value: 0.13172927322785893.
[I 2024-10-15 16:46:24,119] Trial 1 finished with value: 0.12788232657402715 and parameters: {'n_estimators': 500, 'learning_rate': 0.051522122155757064, 'max_depth': 4, 'min_child_weight': 4, 'gamma': 0.02442876929801003, 'subsample': 0.7789847405675518, 'colsample_bytree': 0.9484711036313699, 'reg_alpha': 0.5307381207314802, 'reg_lambda': 0.2556622243886855, 'scale_pos_

Best Parameters: {'n_estimators': 300, 'learning_rate': 0.039507649596015305, 'max_depth': 4, 'min_child_weight': 1, 'gamma': 0.003732989565643707, 'subsample': 0.6321727699000128, 'colsample_bytree': 0.596852817712824, 'reg_alpha': 0.21928812049229313, 'reg_lambda': 1.146932830508916, 'scale_pos_weight': 3, 'booster': 'dart', 'tree_method': 'hist', 'grow_policy': 'depthwise'}
Test Score: 0.9091348052024841
The negative MSLE is 0.12275361273070051


In [109]:
from sklearn.ensemble import VotingRegressor

# Create a voting regressor
voting_regressor = VotingRegressor(
    estimators=[
        ("catboost", best_model_catboost),
        ("xgboost", best_model_xgb),
        ("lightgbm", best_model_LGBM),
    ],
    weights=[3, 1, 1],  # Equal weights for each model
)

# Fit the voting regressor
voting_regressor.fit(X_train, y_train)

# Make predictions
voting_predictions = voting_regressor.predict(X_test)
voting_regressor_score = voting_regressor.score(X_test, y_test)
print(f"Test Score: {voting_regressor_score}")

# Evaluate
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, voting_predictions))}"
)

Test Score: 0.9011776411893856
The negative MSLE is 0.11827143607074196


In [111]:
from sklearn.ensemble import StackingRegressor

# Create a stacking regressor
stacking_regressor = StackingRegressor(
    estimators=[
        ("catboost", best_model_catboost),
        ("xgboost", best_model_xgb),
        ("lightgbm", best_model_LGBM),
    ],
    final_estimator=LinearRegression(),  # Linear seems to work best
)

# Fit the stacking regressor
stacking_regressor.fit(X_train, y_train)

# Make predictions
stacking_predictions = stacking_regressor.predict(X_test)
stacking_regressor_score = stacking_regressor.score(X_test, y_test)
print(f"Test Score: {stacking_regressor_score}")

# Evaluate
print(
    f"The negative MSLE is {np.sqrt(mean_squared_log_error(y_test, stacking_predictions))}"
)

Test Score: 0.9069632467374733
The negative MSLE is 0.11709312088549816


In [98]:
# Here I read in the test file, condition the data and then predict the outomes to a csv file.
path = "test.csv"
df = pd.read_csv(path)
df_index = df.Id
used_model = best_model_catboost

df_test = data_conditioning(dataframe=df)
df_test = df_test[high_corr_vars_list]
df_test_S = pd.DataFrame(scaler.transform(df_test), columns=df_test.columns)
# df_pca_test = pca.transform(df_test_S)

y_prediction = used_model.predict(df_test_S)

output_df = pd.DataFrame({"Id": df_index, "SalePrice": y_prediction})
output_df.set_index("Id").to_csv("Housing_Price_Estimates_TP.csv")

## Ideas moving forward
- Potential for multicolinearity (too many variables look to reduce some) more into feature reduction
- Have missing values, better way to estimate the values of those. (instead of median)
- Potential to feed that into nerual network to improve further?
